## Preliminaries

**Packages**

In [1]:
import subprocess

In [2]:
if 'google.colab' in str(get_ipython()):
    subprocess.run('rm -rf *scripts*', shell=True)
    subprocess.run('wget -q https://raw.githubusercontent.com/briefings/phe/develop/scripts.sh', 
                   shell=True)
    subprocess.run('chmod u+x scripts.sh', shell=True)
    subprocess.run('./scripts.sh', shell=True)

<br>

**Paths**

In [3]:
import os
import pathlib
import sys

In [4]:
if not 'google.colab' in str(get_ipython()):
    
    notebooks = os.getcwd()
    parent = str(pathlib.Path(notebooks).parent)
    sys.path.append(parent)


<br>

**Libraries**

In [5]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

import logging

<br>

**Layout**

In [6]:
sns.set_style("darkgrid")
sns.set_context("poster")
sns.set(font_scale=1)

<br>

**Logging**

In [7]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

<br>
<br>

**Special**

In [8]:
import phe.cases.interface
import phe.baselines.formatting
import phe.algorithms.directories

import phe.highcharts.graphseries
import phe.tableau.graphseries

import config
configurations = config.Config()

<br>
<br>

## Alternative

In [9]:
# Clean-up
phe.algorithms.directories.Directories().exc()

In [10]:
# Get the data, and structure it for graphing
interface = phe.cases.interface.Interface()

for tab in ['admissionstotal', 'admissions85']:

    logger.info('Analysing: \'{}\'\n'.format(tab))

    # Retrieving
    series, institutions, notes = interface.exc(tab=tab)

    # Structuring
    wide, narrow = phe.baselines.formatting.Formatting().exc(blob=series)
    
    # Finally, graphing data ...
    # Note: phe.highcharts.graphseries involves parallel processing
    path = os.path.join(configurations.warehouse, tab)
    phe.tableau.graphseries.GraphSeries(blob=narrow, institutions=institutions, path=path).exc()
    phe.highcharts.graphseries.GraphSeries(blob=wide, institutions=institutions, path=path).exc()

INFO:__main__:Analysing: 'admissionstotal'

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:__main__:Analysing: 'admissions85'



<br>
<br>

## Zip

In [11]:
%%bash

path=archives
rm -rf $path
mkdir $path

# Sections that will be archived
Array=('admissionstotal' 'admissions85')

# Start
echo "starting ..." &> zip.log

for focus in ${Array[*]}
do
echo ${focus} &>> zip.log
zip --junk-paths --recurse-paths ${path}/${focus}.zip warehouse/${focus}/ &>> zip.log
echo -e '\n' &>> zip.log
done
